In [68]:
import glob
import os
import librosa
import numpy as np
import speechpy
from keras import layers, models
from keras.callbacks import TensorBoard
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split

DATA_DIR = "/Users/mbare/Documents/audana-dataset/dataset/Audio/"
SUB_DIRS = ["Truthful", "Deceptive"]


def find_max_mfccs_len(parent_dir,sub_dirs):
    lengths_mfccs = []
    for sub_dir in sub_dirs:
        for file_name in glob.glob(
            os.path.join(
                parent_dir,
                sub_dir,
                "*.wav")):
            X, sample_rate = librosa.load(file_name)
            mfccs = librosa.feature.mfcc(y=X, sr=sample_rate).T
            len_mfccs = mfccs.shape[0]
            lengths_mfccs.append(len_mfccs)
    max_len = sorted(lengths_mfccs,reverse=True)[0]
    print("Max MFCCS: " + str(max_len) + ". Will pad based on this")
    return max_len


def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    mfccs = (librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=20)).T
    mfccs = speechpy.processing.cmvnw(mfccs, win_size=301, variance_normalization=True)
    mfccs = np.mean(mfccs, axis=0)
    return mfccs


def parse_audio_files(parent_dir,sub_dirs):
    features, labels = np.empty((0, 20)), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, 
                                         sub_dir, 
                                         "*.wav")):
            try:
                mfccs = extract_feature(fn)
            except Exception as e:
              print ("Error encountered while parsing file: ", fn)
              continue
            ext_features = np.hstack([mfccs])
            features = np.vstack([features,ext_features])
            labels = np.append(labels, [1 if sub_dir=="Deceptive" else 0])
    return np.array(features), np.array(labels, dtype = np.int)


tensorboard = TensorBoard(log_dir='./logs/run3_20mfccs/', 
                          write_graph=True, 
                          write_images=False)

In [69]:
features, labels = parse_audio_files(DATA_DIR, SUB_DIRS)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42)
model = Sequential()
model.add(Dense(16, kernel_initializer='uniform', activation='relu'))
model.add(Dense(2, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train,
          epochs=200,
          validation_data=(X_test, y_test),
          shuffle=True,
#           verbose=False,
          callbacks=[tensorboard])

Train on 96 samples, validate on 25 samples
Epoch 1/200
96/96 [==============================] - 0s 2ms/step - loss: 0.6932 - acc: 0.4792 - val_loss: 0.6932 - val_acc: 0.4400
Epoch 2/200
96/96 [==============================] - 0s 60us/step - loss: 0.6931 - acc: 0.5208 - val_loss: 0.6933 - val_acc: 0.4400
Epoch 3/200
96/96 [==============================] - 0s 64us/step - loss: 0.6931 - acc: 0.5208 - val_loss: 0.6933 - val_acc: 0.4400
Epoch 4/200
96/96 [==============================] - 0s 64us/step - loss: 0.6931 - acc: 0.5208 - val_loss: 0.6934 - val_acc: 0.4400
Epoch 5/200
96/96 [==============================] - 0s 56us/step - loss: 0.6931 - acc: 0.5208 - val_loss: 0.6934 - val_acc: 0.4400
Epoch 6/200
96/96 [==============================] - 0s 66us/step - loss: 0.6931 - acc: 0.5208 - val_loss: 0.6935 - val_acc: 0.4400
Epoch 7/200
96/96 [==============================] - 0s 67us/step - loss: 0.6930 - acc: 0.5208 - val_loss: 0.6935 - val_acc: 0.4400
Epoch 8/200
96/96 [==============

In [71]:
scores = model.evaluate(X_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

25/25 [==============================] - 0s 42us/step
acc: 44.00%
